In [60]:
import numpy as np

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

In [74]:
import numpy as np

def NOMA(data, tx_params):

    tx_powers = tx_params["powerAllocation"]
    n_users = tx_params["nUsers"]

    data_out = np.zeros(data.shape[1], dtype=np.complex128)

    for i in range(n_users):
        data_out += data[i] * np.sqrt(tx_powers[i])

    return data_out

def Rayleigh_canal(alpha, len) :
    h0 = (1j*np.random.normal(scale=np.sqrt(1/2))+np.random.normal(scale=np.sqrt(1/2)))
    var_xi = (1-alpha**2)
    h = np.zeros(len, dtype=np.complex128)
    for i in range(1, len) :
      xi = (1j*np.random.normal(scale=np.sqrt(var_xi))+np.random.normal(scale=np.sqrt(var_xi)))
      h[i] = alpha*h[i-1] + xi
    return np.concatenate((h0, h), axis=None)

tx_params = {
    "nUsers" : 3,
    "powerAllocation" : np.zeros(3),
    "nCode" : 6,
    "kCode" : 3,
    "codingRate": 1/2,
}

# NOMA power allocation between the nUsers users

# resolve polynomial equation x^3 + 3 x^2 + 2 x^1 = 1
# where x is the power allocation of the first user

Ptot = 1 # total power allocation
polynomial = np.array([1, 3, 2, -1*Ptot])
roots = np.roots(polynomial) # roots of polynomial
x = roots[np.isreal(roots)][0] # keep only real roots

# Example 2: Unequal power allocation
P1 = x
P2 = (x**2 + x)
P3 = (x**3 + 2*x**2 + x)
tx_params["powerAllocation"] = np.array([P1, P2, P3])


In [62]:
data = np.zeros((tx_params["nUsers"], tx_params["nCode"]))

data_out = NOMA(data, tx_params)

In [63]:
nmodbits = 2
binary_source = sionna.utils.BinarySource()
encoder = sionna.fec.conv.ConvEncoder(rate=tx_params["codingRate"])
decoder = sionna.fec.conv.ViterbiDecoder(encoder=encoder, rate=tx_params["codingRate"])
constellation = sionna.mapping.Constellation("qam", nmodbits)
mapper = sionna.mapping.Mapper(constellation=constellation)
demapper = sionna.mapping.Demapper("app",constellation=constellation)

In [75]:
ebn0 = np.arange(0, 11, 1) #dB
snr_db = ebn0 + 10*np.log10(tx_params["codingRate"]) + 10*np.log10(nmodbits) #dB
nb_seq = 1
alpha = 0

for zz in range(nb_seq) :
  for ii in range(len(snr_db)) :

      var_noise = 1/(10**(0.1*snr_db[ii]))

      # NOMA Power allocation
      Ptot = 1 # total power allocation
      polynomial = np.array([1, 3, 2, -1*Ptot/var_noise])
      roots = np.roots(polynomial) # roots of polynomial
      x = roots[np.isreal(roots)][0] # keep only real roots

      P1 = x
      P2 = (x**2 + x)
      P3 = (x**3 + 2*x**2 + x)
      tx_params["powerAllocation"] = var_noise*np.array([P1, P2, P3])
      #print(P1/var_noise, P2/(P1+var_noise), P3/(P1+P2+var_noise))

      data = np.zeros((tx_params["nUsers"], tx_params["nCode"]))
      b = binary_source([tx_params["nUsers"], tx_params["kCode"]])
      c = encoder(b)
      x = mapper(c)
      data = x.numpy()

      data_out = NOMA(data, tx_params)
      h = Rayleigh_canal(alpha, len(data_out))


